<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Import-the-data" data-toc-modified-id="Import-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import the data</a></span></li><li><span><a href="#Define-independent-and-target-variables" data-toc-modified-id="Define-independent-and-target-variables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Define independent and target variables</a></span></li><li><span><a href="#Fit-the-model" data-toc-modified-id="Fit-the-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fit the model</a></span></li><li><span><a href="#Analyze-results" data-toc-modified-id="Analyze-results-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Analyze results</a></span></li><li><span><a href="#Level-up-(Optional)" data-toc-modified-id="Level-up-(Optional)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Level up (Optional)</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [2]:
# Import the data
import pandas as pd 

df = pd.read_csv("titanic.csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [3]:
# Total number of people who survived/didn't survive
print("Number of people who didn't survive:", (df['Survived'] == 0).sum())
print("Number of people who survived:", (df['Survived'] == 1).sum())

Number of people who didn't survive: 549
Number of people who survived: 342


Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [4]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
dummy_dataframe = pd.get_dummies(df[relevant_columns], columns=['Pclass', 'Sex', 'Embarked'], drop_first=True).astype(float)


dummy_dataframe.shape

(891, 9)

Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [5]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(714, 9)

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [6]:
# Split the data into X and y
y = dummy_dataframe['Survived']
X = dummy_dataframe.drop('Survived', axis=1)

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [7]:
# Build a logistic regression model using statsmodels

# import the necessary library 
import statsmodels.api as sm

# Build a logistic regression model using statsmodels
X = sm.add_constant(X)  # Add intercept term
model = sm.Logit(y, X)
result = model.fit()


Optimization terminated successfully.
         Current function value: 0.443266
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [8]:
# Summary table
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      705
Method:                           MLE   Df Model:                            8
Date:                Fri, 06 Jun 2025   Pseudo R-squ.:                  0.3437
Time:                        21:02:12   Log-Likelihood:                -316.49
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.889e-67
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4362      0.534      8.302      0.000       3.389       5.484
Age           -0.0431      0.008     -5.190      0.000      -0.059      -0.027
SibSp         -0.3804      0.125     -3.041      0.0

In [9]:
# Your comments here
"""
P-value analysis:
- const (intercept): If p-value < 0.05, the intercept is significant, indicating a baseline log-odds when all predictors are zero.
- Age: If p-value < 0.05, age significantly affects survival probability. Negative coefficient suggests older passengers were less likely to survive.
- SibSp: If p-value < 0.05, number of siblings/spouses aboard is significant. Likely negative, as larger families may have lower survival chances.
- Fare: If p-value < 0.05, fare is significant. Positive coefficient expected, as higher fares (higher class) correlate with better survival odds.
- Pclass_2, Pclass_3: If p-values < 0.05, these classes significantly differ from Pclass_1 (reference). Negative coefficients expected, as lower classes had lower survival rates.
- Sex_male: If p-value < 0.05, gender is significant. Strong negative coefficient expected, as males were less likely to survive than females.
- Embarked_Q, Embarked_S: If p-values < 0.05, embarkation ports differ significantly from Cherbourg (reference). Coefficients may vary based on data.

Based on typical Titanic data, Sex_male, Pclass_3, and Age are likely highly significant (p < 0.05) with large coefficients, indicating strong influence on survival.
"""

'\nP-value analysis:\n- const (intercept): If p-value < 0.05, the intercept is significant, indicating a baseline log-odds when all predictors are zero.\n- Age: If p-value < 0.05, age significantly affects survival probability. Negative coefficient suggests older passengers were less likely to survive.\n- SibSp: If p-value < 0.05, number of siblings/spouses aboard is significant. Likely negative, as larger families may have lower survival chances.\n- Fare: If p-value < 0.05, fare is significant. Positive coefficient expected, as higher fares (higher class) correlate with better survival odds.\n- Pclass_2, Pclass_3: If p-values < 0.05, these classes significantly differ from Pclass_1 (reference). Negative coefficients expected, as lower classes had lower survival rates.\n- Sex_male: If p-value < 0.05, gender is significant. Strong negative coefficient expected, as males were less likely to survive than females.\n- Embarked_Q, Embarked_S: If p-values < 0.05, embarkation ports differ sign

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [10]:
# Your code here

# Create a new model with influential features (based on p-values < 0.05 from initial model)
# Assuming Sex_male, Pclass_3, Age are significant (adjust based on actual summary)
significant_columns = ['Age', 'Pclass_3', 'Sex_male', 'Survived']
level_up_df = dummy_dataframe[significant_columns]
y_level_up = level_up_df['Survived']
X_level_up = sm.add_constant(level_up_df.drop('Survived', axis=1))
level_up_model = sm.Logit(y_level_up, X_level_up)
level_up_result = level_up_model.fit()

# Summary table for level-up model
print(level_up_result.summary())

Optimization terminated successfully.
         Current function value: 0.469678
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      710
Method:                           MLE   Df Model:                            3
Date:                Fri, 06 Jun 2025   Pseudo R-squ.:                  0.3046
Time:                        21:03:59   Log-Likelihood:                -335.35
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 2.168e-63
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7423      0.310      8.845      0.000       2.135       3.350
Age           -0.0268      0.

In [ ]:
# Your comments here
"""
Level-up model performance:
- The reduced model uses only significant predictors (e.g., Sex_male, Pclass_3, Age), simplifying interpretation and potentially improving robustness.
- If p-values remain < 0.05, these features are still significant, confirming their importance.
- Compare Pseudo R-squared: If similar to or slightly lower than the full model, the reduced model retains most explanatory power with fewer variables.
- Log-Likelihood: A slightly lower (less negative) value than the full model indicates a minor loss in fit, but simpler models are preferred if performance is comparable.
- This model is more interpretable and less prone to overfitting due to fewer features, but may miss some predictive power from excluded variables like Fare or Embarked.
"""

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!